## Reshaping and Pivot Tables

### Reshaping by pivoting DataFrame objects

Create a random dataset

In [4]:
import numpy as np
import pandas as pd
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())
df.head()

,date,variable,value
0,2000-01-03,A,0.529687
1,2000-01-04,A,0.700495
2,2000-01-05,A,0.114713
3,2000-01-03,B,-2.313602
4,2000-01-04,B,-1.703037


To select out everything for variable A we could do:

In [5]:
df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,0.529687
1,2000-01-04,A,0.700495
2,2000-01-05,A,0.114713


But suppose we wish to do time series operations with the variables. A better representation would be where the columns are the unique variables and an index of dates identifies individual observations. To reshape the data into this form, use the pivot function

In [6]:
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,0.529687,-2.313602,0.118358,1.388616
2000-01-04,0.700495,-1.703037,2.079999,1.212176
2000-01-05,0.114713,-1.262516,-1.652672,-0.571822


If the values argument is omitted, and the input DataFrame has more than one column of values which are not used as column or index inputs to pivot, then the resulting “pivoted” DataFrame will have hierarchical columns whose topmost level indicates the respective value column:

In [8]:
df['value2'] = df['value'] * 2
pivoted = df.pivot('date', 'variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.529687 -2.313602  0.118358  1.388616  1.059374 -4.627205   
2000-01-04  0.700495 -1.703037  2.079999  1.212176  1.400990 -3.406073   
2000-01-05  0.114713 -1.262516 -1.652672 -0.571822  0.229426 -2.525031   

                                
variable           C         D  
date                            
2000-01-03  0.236715  2.777231  
2000-01-04  4.159999  2.424351  
2000-01-05 -3.305345 -1.143643

You of course can then select subsets from the pivoted DataFrame:

In [9]:
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,1.059374,-4.627205,0.236715,2.777231
2000-01-04,1.400990,-3.406073,4.159999,2.424351
2000-01-05,0.229426,-2.525031,-3.305345,-1.143643


Closely related to the pivot function are the related stack and unstack functions currently available on Series and DataFrame. These functions are designed to work together with MultiIndex objects (see the section on hierarchical indexing). Here are essentially what these functions do:    

* stack: “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an index with a new inner-most level of row labels.    
* unstack: inverse operation from stack: “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.    

The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing section:

In [11]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
   ...:                      'foo', 'foo', 'qux', 'qux'],
   ...:                     ['one', 'two', 'one', 'two',
   ...:                      'one', 'two', 'one', 'two']]))
   ...: 
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
df2 = df[:4]
df2

A         B
first second                    
bar   one    -0.018307 -0.449205
      two     0.105778 -0.759560
baz   one    -0.338284  1.795714
      two     0.411483  0.015669

The stack function “compresses” a level in the DataFrame’s columns to produce either:    

* A Series, in the case of a simple column Index
* A DataFrame, in the case of a MultiIndex in the columns    

If the columns have a MultiIndex, you can choose which level to stack. The stacked level becomes the new lowest level in a MultiIndex on the columns:

In [13]:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -0.018307
               B   -0.449205
       two     A    0.105778
               B   -0.759560
baz    one     A   -0.338284
               B    1.795714
       two     A    0.411483
               B    0.015669
dtype: float64

With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of stack is unstack, which by default unstacks the last level:

In [14]:
stacked.unstack()


A         B
first second                    
bar   one    -0.018307 -0.449205
      two     0.105778 -0.759560
baz   one    -0.338284  1.795714
      two     0.411483  0.015669

In [15]:
stacked.unstack(1)

second        one       two
first                      
bar   A -0.018307  0.105778
      B -0.449205 -0.759560
baz   A -0.338284  0.411483
      B  1.795714  0.015669

In [16]:
stacked.unstack(0)

first          bar       baz
second                      
one    A -0.018307 -0.338284
       B -0.449205  1.795714
two    A  0.105778  0.411483
       B -0.759560  0.015669

If the indexes have names, you can use the level names instead of specifying the level numbers:

In [17]:
stacked.unstack('second')

second        one       two
first                      
bar   A -0.018307  0.105778
      B -0.449205 -0.759560
baz   A -0.338284  0.411483
      B  1.795714  0.015669